# Object Detection: People, Bikes, and Cars
Below we will do the following:
1. Setup the training environment.
2. Load images of cars, bikes and people and their corresponding bounding boxes.
2. Build an object detection model for identifying the location of cars, bikes and people in images.
3. Convert the model to CoreML and download it.

The example is based on [Turi Create's Object Detection model](https://github.com/apple/turicreate/tree/master/userguide/object_detection).

## Environment Setup
Below we:
- Remove existing cuda versions and install cuda 9.1 
- Install TuriCreate 5.2 and some specific dependencies to get GPU training capabilities.
- We upgrade numpy so that we can export to coreml at the end.

In [0]:
# we use /dev/null to suppress much of the output and 2>&1 to suppress errors
!rm -rf /usr/local/cuda* >/dev/null

# fetch cuda 9.1 from nvidia and install it
!wget https://developer.nvidia.com/compute/cuda/9.1/Prod/local_installers/cuda-repo-ubuntu1604-9-1-local_9.1.85-1_amd64 >/dev/null
!sudo dpkg -i cuda-repo-ubuntu1604-9-1-local_9.1.85-1_amd64 >/dev/null 2>&1
!sudo apt-key add /var/cuda-repo-9-1-local/7fa2af80.pub 
!sudo apt-get update >/dev/null 2>&1
!sudo apt-get install cuda-9.1 >/dev/null 2>&1

!nvcc --version

In [0]:
!pip install skafos
!pip install turicreate==5.2 2>&1
!pip uninstall -y mxnet 
!pip install mxnet-cu91==1.1.0 2>&1

In [0]:
!pip install numpy --upgrade 2>&1

## Data Preparation and Model Training
The data for this example are 1096 images: 420 cars, 365 bikes, and 311 people. You can find the original dataset [here](https://lear.inrialpes.fr/people/marszalek/data/ig02/).

Once loaded, the data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. By default, this runtime environment will use Python 3 GPU backend instance.

In [0]:
# Import libraries
import tarfile
import urllib
import turicreate as tc
from skafos import summary, models
import os
tc.config.set_num_gpus(1)

- We will be uploading this newly trained Object Detection model Skafos.

- If you haven't already done so, check out [Skafos](dashboard.skafos.ai) and create an App.

- If you have, navigate to the top right corner where you can find your **API Token** under My Account

In [0]:
# Set your API Token first for repeated use
os.environ["SKAFOS_API_TOKEN"] = "<YOUR-SKAFOS-API-TOKEN>"

In [0]:
# Below you will see a list of apps and models you have access to.
summary()

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
## Load the data
data =  tc.SFrame('ig02.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

In [0]:
train_data.head()

In [0]:
# Create an object detection model
model = tc.object_detector.create(
    dataset=train_data,
    max_iterations=10 # to improve model accuracy, remove the max_iterations argument and TuriCreate will tune to an appropriate number
)

# Object Detector Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.object_detector.create.html#turicreate.object_detector.create

In [0]:
model.evaluate(test_data)

## Model Export and Upload
We convert the model to CoreML format so that it can run on an iOS device.

We then upload it to skafos using the Skafos Python SDK.

In [0]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

In [0]:
# You can retrieve this info with skafos.summary()
org_name = "<YOUR-SKAFOS-ORG-NAME>"    # Example: "mike-gmail-com-467h2"
app_name = "<YOUR-SKAFOS-APP-NAME>"    # Example: "Recommender-App"
model_name = "<YOUR-MODEL-NAME>"       # Example: "RecommenderModel"

# Upload model version to Skafos
model_upload_result = models.upload_version(
    files=coreml_model_name,
    org_name=org_name,
    app_name=app_name,
    model_name=model_name
)